In [28]:
import pandas as pd
import datetime
import numpy as np
import pandas as pd
import pickle

In [2]:
PATH = "./train_data.csv"

In [3]:
df = pd.read_csv(PATH)

In [4]:
df = df[(df.lat > 59.831191) & (df.lat < 60.039332) & (df.lon > 30.142969) & (df.lon < 30.515771)]

In [5]:
def get_all_timestamps(min_v: int, max_v: int, name: str = "timestamp") -> pd.DataFrame:
    delta = datetime.timedelta(hours=1)
    min_v = datetime.datetime.fromtimestamp(min_v)
    max_v = datetime.datetime.fromtimestamp(max_v)
    value = min_v
    values = [value, ]
    while value != max_v:
        value += delta
        values.append(value)
    values = list(map(lambda x: x.timestamp(), values))
    df = pd.DataFrame()
    df[name] = values
    return df

In [6]:
all_timestamps = get_all_timestamps(df.timestamp.min(), df.timestamp.max())

In [12]:
points = []
groups = []
n_records = []
for point, g in df.groupby("point"):
    points.append(point)
    g = g.groupby("timestamp").apply(len).to_frame("n_pubs").reset_index(drop=False)
    n_records.append(len(g))
    g = g.merge(all_timestamps, how="outer", on="timestamp")
    g = g.fillna(0)
    g = g.set_index("timestamp").sort_index()["n_pubs"]
    groups.append(g)

In [13]:
pd.Series(n_records).describe()

count    6658.000000
mean      532.834935
std      1198.525750
min         1.000000
25%        17.000000
50%        79.000000
75%       371.000000
max      9119.000000
dtype: float64

In [17]:
selected_idxes = np.where(np.array(n_records) >= 371)[0]

In [18]:
points = [points[i] for i in selected_idxes]
groups = [groups[i] for i in selected_idxes]

In [29]:
with open("data/selected_data.pickle", "wb") as f:
    pickle.dump({"points": points, "groups": groups}, f)